In [30]:
import os
import ast
import pandas as pd
import json

# Comparison of extraction with literature dataset

Notebook to compare results from automated extraction with manually compiled dataset from authors of https://doi.org/10.1021/acs.jcim.2c01007

In [33]:
# Importing literature dataset as csv to pandas dataframe
# path = '/Users/pnt17/Library/CloudStorage/OneDrive-ImperialCollegeLondon/MRes_project_data'
path = r'C:\Users\Piotr\OneDrive - Imperial College London\MRes_project_data'
filename = 'ML_CD_database_test1_test2_csv.csv'
lit_data = pd.read_csv(os.path.join(path, filename))
print(lit_data.shape)
lit_data.head()

(423, 23)


,Example ID,DOI,Color,citric acid,urea,ethylenediamine,HCl,ammonium,NaOH,boric acid,...,Reaction method,Solvent,pH,Reaction temperatrue( C ),reaction time (min),purification method,particle size (TEM),Lattice obrserved or not,absorption,Main Peak (in water)
0,1,10.1016/j.snb.2018.09.031,blue,1.0000,0.0,0.0,0.000,0,0.0,0.0,...,solvothermal treatment,"ethanol, formamide",acidic,423.15,300.0,centrifuge,4.50,Y,435.0,450
1,2,10.1039/c9ra04241e,blue,0.1560,1.0,0.0,0.003,0,0.0,0.0,...,hydrothermal treatment,water,acidic,433.15,240.0,centrifuge,2.81,Y,493.0,445
2,3,10.1007/s12274-019-2569-3,blue,1.0000,0.0,0.0,0.000,0,0.0,0.0,...,solvothermal treatment,ethanol,acidic,473.15,420.0,"filter, dialysis",4.00,N,370.0,440
3,4,10.1016/j.jcis.2018.03.021,blue,0.0825,0.0,0.0,0.000,1,0.0,0.0,...,microwave,water,neutral,473.15,7.0,centrifuge,1.80,Y,285.0,450
4,5,10.1039/C6RA20562C,blue,1.0000,0.0,0.0,0.000,0,0.0,0.0,...,hydrothermal treatment,water,acidic,433.15,360.0,dialysis,1.62,N,245.0,415


In [34]:
len(set(lit_data['DOI'].to_list()))

169

In [35]:
# Keeping only hydrothermal/solvothermal reactions from literature dataset
lit_data_2 = lit_data.loc[(lit_data['Reaction method'] == 'hydrothermal treatment') | (lit_data['Reaction method'] == 'solvothermal treatment')]
print(lit_data_2.shape)

(267, 23)


In [36]:
# Removing unnecessary columns
remove_col = ['Example ID', 'Color', 'pH', 'purification method', 'particle size (TEM)', 'Lattice obrserved or not', 'absorption', 'Main Peak (in water)']
lit_data_2 = lit_data_2.drop(columns = remove_col)
print(lit_data_2.shape)

(267, 15)


In [38]:
lit_dois = lit_data_2['DOI'].tolist()
len(set(lit_dois))

101

Literature dataset has repeating DOIs for different reaction conditions and dois from publishers outside of set chosen for extraction

In [21]:
#Filtering lit_dois for duplicates and keeping only the ones from select publishers
lit_dois = lit_dois[:len(lit_dois)-4]

In [25]:
prefixes = ['10.1016', '10.1039', '10.1007', '10.1021', '10.1002', '10.3390', '10.1080', '10.1038']
lit_dois_filtered = [doi for doi in lit_dois if doi.split('/',1)[0] in prefixes]

In [26]:
lit_dois_filtered = list(set(lit_dois_filtered))
len(lit_dois_filtered)

84

In [10]:
#Getting relevant extracted data
path = '/Users/pnt17/Library/CloudStorage/OneDrive-ImperialCollegeLondon/MRes_project_data/ceder_extract_data'
filename = 'relevant_data.json'
with open(os.path.join(path, filename),'r', encoding = 'utf-8') as file:
    rel_data = json.load(file)

In [12]:
rel_dois = [entry['doi'] for entry in rel_data]

In [31]:
# Finding which lit_dois are in rel_dois
matches = []
for doi in lit_dois_filtered:
    for doi_2 in rel_dois:
        if doi_2.lower().startswith(doi.lower()):
            matches.append(doi_2)

In [32]:
len(matches)

34

In [37]:
#Checking for matches in doi search results
path = '/Users/pnt17/Library/CloudStorage/OneDrive-ImperialCollegeLondon/MRes_project_data/doi_sem_scholar_2'
filename = 'doi_select.txt'
with open(os.path.join(path, filename),'r', encoding = 'utf-8') as file:
    doi_search_result = file.read().splitlines()


In [38]:
search_matches = []
for doi in lit_dois_filtered:
    for doi_2 in doi_search_result:
        if doi_2.lower() == doi.lower():
            search_matches.append(doi_2)
len(search_matches)

47

In [39]:
differences = list(set(lit_dois_filtered) - set(search_matches))
differences

['10.1016/j.snb.2018.09.031',
 '10.1039/D0EE02235G',
 '10.1002/anie.201301114',
 '10.1016/j.optmat.2017.08.032',
 '10.1039/c9nj03874d',
 '10.1002/slct.201803716',
 '10.1016/j.jlumin.2016.01.036',
 '10.1039/c9nr08663c',
 '10.1038/s41598-019-50922-6',
 '10.1039/c9tc03481a',
 '10.1021/acs.chemmater.7b04446',
 '10.1007/s10895-016-1867-3',
 '10.1039/c5ra27172j',
 '10.1021/acsanm.9b01080',
 '10.1007/s12274-017-1838-2',
 '10.1021/acsanm.0c02547',
 '10.1039/c5ra04653j',
 '10.1039/C6RA20562C',
 '10.1016/j.msec.2017.05.049',
 '10.1002/advs.201903525',
 '10.1039/c6ra02546c',
 '10.1021/acssuschemeng.8b01618',
 '10.1039/D1NA00286D',
 '10.1016/j.enconman.2017.10.010',
 '10.1016/j.carbon.2020.11.030',
 '10.1016/j.jallcom.2017.01.124',
 '10.1039/c6cp05813b',
 '10.1016/j.molliq.2017.06.106',
 '10.1016/j.chemphys.2019.110503',
 '10.1016/j.cej.2020.124654',
 '10.1039/c3nr04402e ',
 '10.1039/c5ay01715g',
 '10.1002/chem.202000763',
 '10.1016/j.electacta.2019.02.012',
 '10.1002/adma.201705913',
 '10.1016/j.

The literature dataset contains data from 84 articles from the selected publishers. The semantic scholar search retrieved only 47 of those articles and only from 34 articles was data extracted. Actually 33 (two paragraphs from one article) 

## Comparing data for matching DOIs

In [41]:
#Creating dataframe for comparison
compare_df = pd.DataFrame(columns = ['DOI', 'extracted precursors', 'annotated precursors', 'extracted rxn times', 'annotated rxn times', 'extracted rxn temps', 'annotated rxn_temps'])

In [42]:
compare_df['DOI'] = matches

In [43]:
# Converting lit dataset to cvs with temps in celsius and time in hours
def temp_to_celsius(value):
    return value-273.15
def time_to_hours(value):
    return round(value/60,2)
lit_data_2['reaction time (min)'] = lit_data_2['reaction time (min)'].apply(time_to_hours)
lit_data_2['Reaction temperatrue( C )'] = lit_data_2['Reaction temperatrue( C )'].apply(temp_to_celsius)

In [44]:
path = path = '/Users/pnt17/Library/CloudStorage/OneDrive-ImperialCollegeLondon/MRes_project_data/ceder_extract_data'
lit_data_2.to_csv(os.path.join(path,'lit_dataset.csv'), index = False)
compare_df.to_csv(os.path.join(path, 'comparison.csv'), index = False)

In [15]:
#Importing csv with data from extraction and literature dataset to pandas
path = r'C:\Users\Piotr\OneDrive - Imperial College London\MRes_project_data\ceder_extract_data'
filename = 'comparison.csv'
comparison_df = pd.read_csv(os.path.join(path, filename))
for col in comparison_df.columns[1:]:
    comparison_df[col] = comparison_df[col].apply(ast.literal_eval)
comparison_df.head(10)

,DOI,extracted precursors,annotated precursors,extracted rxn times,annotated rxn times,extracted rxn temps,annotated rxn_temps
0,10.3390/molecules25102320,[ethylenediamine],"[citric acid, ethylenediamine]",[],[4],[],[175]
1,10.1039/D0EE02235G_no_1,"[citric acid, urea]","[citric acid, urea]",[6],[6],[160],[160]
2,10.1038/srep05294,"[citric acid, NaOH]","[citric acid, urea, NaOH]",[4],"[2, 4, 6, 8, 24]",[160],[160]
3,10.1007/s00604-018-2999-8,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[220],[220]
4,10.1007/S10404-016-1825-Z,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200]
5,10.1007/s00604-019-3933-4,"[citric acid, urea, CDs, CTAB]","[citric acid, urea]",[8],[8],[180],[180]
6,10.1039/C5RA04653J,"[citric acid, urea]","[citric acid, urea]",[4],"[2, 4, 6, 8, 10]",[180],"[140, 160, 180, 200, 220]"
7,10.1039/C6RA02546C_no_1,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200]
8,10.1039/C5AY01715G,"[citric acid, urea]","[citric acid, urea]",[24],[24],[180],[180]
9,10.1007/s12161-020-01700-y,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[200],[200]


In [19]:
ext_pre = comparison_df['extracted precursors'].to_list()
ann_pre = comparison_df['annotated precursors'].to_list()
pre_val = []
for i, text in enumerate(ann_pre):
    vals = []
    for pre in text:
        if pre in ext_pre[i]:
            vals.append('TP')
        if pre not in ext_pre[i]:
            vals.append('FN')
    for material in ext_pre[i]:
        if material not in text:
            vals.append('FP')
    pre_val.append(vals)
comparison_df['precursor_tags'] = pre_val
comparison_df.head(10)

,DOI,extracted precursors,annotated precursors,extracted rxn times,annotated rxn times,extracted rxn temps,annotated rxn_temps,precursor_tags
0,10.3390/molecules25102320,[ethylenediamine],"[citric acid, ethylenediamine]",[],[4],[],[175],"[FN, TP]"
1,10.1039/D0EE02235G_no_1,"[citric acid, urea]","[citric acid, urea]",[6],[6],[160],[160],"[TP, TP]"
2,10.1038/srep05294,"[citric acid, NaOH]","[citric acid, urea, NaOH]",[4],"[2, 4, 6, 8, 24]",[160],[160],"[TP, FN, TP]"
3,10.1007/s00604-018-2999-8,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[220],[220],"[TP, TP]"
4,10.1007/S10404-016-1825-Z,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]"
5,10.1007/s00604-019-3933-4,"[citric acid, urea, CDs, CTAB]","[citric acid, urea]",[8],[8],[180],[180],"[TP, TP, FP, FP]"
6,10.1039/C5RA04653J,"[citric acid, urea]","[citric acid, urea]",[4],"[2, 4, 6, 8, 10]",[180],"[140, 160, 180, 200, 220]","[TP, TP]"
7,10.1039/C6RA02546C_no_1,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]"
8,10.1039/C5AY01715G,"[citric acid, urea]","[citric acid, urea]",[24],[24],[180],[180],"[TP, TP]"
9,10.1007/s12161-020-01700-y,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[200],[200],"[TP, TP]"


In [20]:
ext_time = comparison_df['extracted rxn times'].to_list()
ann_time = comparison_df['annotated rxn times'].to_list()
time_val = []
for i, text in enumerate(ann_time):
    vals = []
    for time in text:
        if time in ext_time[i]:
            vals.append('TP')
        if time not in ext_time[i]:
            vals.append('FN')
    for value in ext_time[i]:
        if value not in text:
            vals.append('FP')
    time_val.append(vals)
comparison_df['time_tags'] = time_val
comparison_df.head(10)

,DOI,extracted precursors,annotated precursors,extracted rxn times,annotated rxn times,extracted rxn temps,annotated rxn_temps,precursor_tags,time_tags
0,10.3390/molecules25102320,[ethylenediamine],"[citric acid, ethylenediamine]",[],[4],[],[175],"[FN, TP]",[FN]
1,10.1039/D0EE02235G_no_1,"[citric acid, urea]","[citric acid, urea]",[6],[6],[160],[160],"[TP, TP]",[TP]
2,10.1038/srep05294,"[citric acid, NaOH]","[citric acid, urea, NaOH]",[4],"[2, 4, 6, 8, 24]",[160],[160],"[TP, FN, TP]","[FN, TP, FN, FN, FN]"
3,10.1007/s00604-018-2999-8,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[220],[220],"[TP, TP]",[TP]
4,10.1007/S10404-016-1825-Z,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]",[TP]
5,10.1007/s00604-019-3933-4,"[citric acid, urea, CDs, CTAB]","[citric acid, urea]",[8],[8],[180],[180],"[TP, TP, FP, FP]",[TP]
6,10.1039/C5RA04653J,"[citric acid, urea]","[citric acid, urea]",[4],"[2, 4, 6, 8, 10]",[180],"[140, 160, 180, 200, 220]","[TP, TP]","[FN, TP, FN, FN, FN]"
7,10.1039/C6RA02546C_no_1,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]",[TP]
8,10.1039/C5AY01715G,"[citric acid, urea]","[citric acid, urea]",[24],[24],[180],[180],"[TP, TP]",[TP]
9,10.1007/s12161-020-01700-y,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[200],[200],"[TP, TP]",[TP]


In [22]:
ext_temp = comparison_df['extracted rxn temps'].to_list()
ann_temp = comparison_df['annotated rxn_temps'].to_list()
temp_val = []
for i, text in enumerate(ann_temp):
    vals = []
    for temp in text:
        if temp in ext_temp[i]:
            vals.append('TP')
        if temp not in ext_temp[i]:
            vals.append('FN')
    for value in ext_temp[i]:
        if value not in text:
            vals.append('FP')
    temp_val.append(vals)
comparison_df['temp_tags'] = temp_val
comparison_df.head(10)

,DOI,extracted precursors,annotated precursors,extracted rxn times,annotated rxn times,extracted rxn temps,annotated rxn_temps,precursor_tags,time_tags,temp_tags
0,10.3390/molecules25102320,[ethylenediamine],"[citric acid, ethylenediamine]",[],[4],[],[175],"[FN, TP]",[FN],[FN]
1,10.1039/D0EE02235G_no_1,"[citric acid, urea]","[citric acid, urea]",[6],[6],[160],[160],"[TP, TP]",[TP],[TP]
2,10.1038/srep05294,"[citric acid, NaOH]","[citric acid, urea, NaOH]",[4],"[2, 4, 6, 8, 24]",[160],[160],"[TP, FN, TP]","[FN, TP, FN, FN, FN]",[TP]
3,10.1007/s00604-018-2999-8,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[220],[220],"[TP, TP]",[TP],[TP]
4,10.1007/S10404-016-1825-Z,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]",[TP],[TP]
5,10.1007/s00604-019-3933-4,"[citric acid, urea, CDs, CTAB]","[citric acid, urea]",[8],[8],[180],[180],"[TP, TP, FP, FP]",[TP],[TP]
6,10.1039/C5RA04653J,"[citric acid, urea]","[citric acid, urea]",[4],"[2, 4, 6, 8, 10]",[180],"[140, 160, 180, 200, 220]","[TP, TP]","[FN, TP, FN, FN, FN]","[FN, FN, TP, FN, FN]"
7,10.1039/C6RA02546C_no_1,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[6],[6],[200],[200],"[TP, TP]",[TP],[TP]
8,10.1039/C5AY01715G,"[citric acid, urea]","[citric acid, urea]",[24],[24],[180],[180],"[TP, TP]",[TP],[TP]
9,10.1007/s12161-020-01700-y,"[citric acid, ethylenediamine]","[citric acid, ethylenediamine]",[5],[5],[200],[200],"[TP, TP]",[TP],[TP]


In [25]:
precursor_vals = [item for sublist in comparison_df['precursor_tags'].to_list() for item in sublist]
time_vals = [item for sublist in comparison_df['time_tags'].to_list() for item in sublist]
temp_vals = [item for sublist in comparison_df['temp_tags'].to_list() for item in sublist]

In [26]:
# Calculating metrics for precursor
TP = precursor_vals.count('TP')
FP = precursor_vals.count('FP')
FN = precursor_vals.count('FN')
precision = TP / (TP+FP)
recall = TP / (TP+FN)
F1 = (2*precision*recall)/(precision+recall)
print(f'The precision is: {precision:.2f}')
print(f'The recall is: {recall:.2f}')
print(f'The F1 score is: {F1:.2f}')

The precision is: 0.85
The recall is: 0.89
The F1 score is: 0.87


In [27]:
# Calculating metrics for rxn time
TP = time_vals.count('TP')
FP = time_vals.count('FP')
FN = time_vals.count('FN')
precision = TP / (TP+FP)
recall = TP / (TP+FN)
F1 = (2*precision*recall)/(precision+recall)
print(f'The precision is: {precision:.2f}')
print(f'The recall is: {recall:.2f}')
print(f'The F1 score is: {F1:.2f}')

The precision is: 0.97
The recall is: 0.60
The F1 score is: 0.74


In [28]:
# Calculating metrics for rxn temp
TP = temp_vals.count('TP')
FP = temp_vals.count('FP')
FN = temp_vals.count('FN')
precision = TP / (TP+FP)
recall = TP / (TP+FN)
F1 = (2*precision*recall)/(precision+recall)
print(f'The precision is: {precision:.2f}')
print(f'The recall is: {recall:.2f}')
print(f'The F1 score is: {F1:.2f}')

The precision is: 1.00
The recall is: 0.64
The F1 score is: 0.78
